# Notebook 04 – RAG científico con arXiv y embeddings

En este notebook se implementa una arquitectura RAG (Retrieval-Augmented Generation)
para mejorar la calidad del contenido científico divulgativo.

La información se obtiene de artículos científicos de arXiv
y se utiliza como contexto para el modelo de lenguaje.


## Arquitectura RAG en este proyecto

1. Loader: obtiene documentos de arXiv
2. Splitter: divide los textos en fragmentos
3. Embeddings: convierte texto en vectores
4. Vector Store: almacena los vectores
5. Retriever: recupera contexto relevante
6. LLM: genera la respuesta final


!pip install arxiv langchain-community chromadb tiktoken  || instalamos las dependencias (done)


(Cargar documentos de arXiv)

📁 backend/app/domain/rag/arxiv_loader.py

In [ ]:
from langchain_community.document_loaders import ArxivLoader

loader = ArxivLoader(
    query="quantum computing",
    max_docs=5
)

documents = loader.load()


## ¿Por qué dividir los documentos?

Los LLMs no manejan textos largos de forma eficiente.
Dividir los documentos permite:
- Mejor búsqueda semántica
- Menos ruido
- Respuestas más precisas


📁 backend/app/domain/rag/text_splitter.py

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = splitter.split_documents(documents)


## Embeddings

Un embedding es una representación numérica de un texto.
Permite comparar textos por significado, no por palabras exactas.


Código (Embeddings con Groq-compatible)

⚠️ Groq NO genera embeddings, así que usamos HuggingFace local/gratuito.

📁 backend/app/domain/rag/embeddings.py

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


📁 backend/app/domain/rag/vectorstore.py

In [ ]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./data/vectorstore"
)

## Retriever

El retriever se encarga de buscar los fragmentos más relevantes
para una pregunta concreta.


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

📁 backend/app/services/rag_service.py

In [ ]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=os.getenv("GROQ_API_KEY")
)

query = "Explica la computación cuántica para el público general"

docs = retriever.get_relevant_documents(query)

context = "\n\n".join([doc.page_content for doc in docs])

final_prompt = f"""
Usa el siguiente contexto científico para responder de forma divulgativa:

{context}

Pregunta:
{query}
"""

response = llm.invoke(final_prompt)
print(response.content)


Este notebook implementa una arquitectura RAG completa
para generación de contenido científico divulgativo,
mejorando la calidad, precisión y fiabilidad del contenido generado.
